In [114]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-white')

from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

rng = 0


In [115]:
# Loading of dataset, with data preprocessing
train_features = pd.read_csv('./train_features_modified.csv', index_col=[0, 1, 2])
train_labels = pd.read_csv('./dengue_labels_train.csv', index_col=[0, 1, 2])
test_features = pd.read_csv('./test_features_modified.csv', index_col=[0, 1, 2])

# drop week_start_date column
train_features = train_features.iloc[:,1:]
test_features = train_features.iloc[:,1:]


In [116]:
# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_train_labels = train_labels.loc['sj']
sj_test_features = test_features.loc['sj']

# Separate data for Iquitos
iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc['iq']
iq_test_features = test_features.loc['iq']


## Clustering


In [120]:
kmeans_sj = KMeans(random_state=rng).fit(sj_train_features)
clusters_sj = kmeans_sj.predict(sj_train_features)


[5 5 5 5 1 1 1 4 3 3 5 5 5 5 5 0 3 3 5 3 4 4 3 0 0 2 2 4 3 5 5 5 1 5 1 1 1
 1 1 1 1 1 1 5 1 1 1 1 1 1 5 5 5 5 1 5 5 5 5 5 3 3 5 5 5 3 3 5 5 5 3 3 3 0
 3 3 5 5 3 3 4 3 0 5 5 1 1 1 4 3 3 1 1 1 1 1 1 1 1 1 3 3 4 3 5 1 1 0 4 2 2
 0 3 5 1 5 3 3 3 3 4 3 3 5 3 0 4 3 3 5 3 3 0 0 0 0 4 3 5 1 5 1 1 1 5 5 5 1
 1 1 1 1 1 1 1 3 3 0 0 5 0 0 3 5 0 3 3 0 3 0 5 5 3 5 3 5 5 3 3 3 5 1 5 5 5
 1 0 5 5 1 5 5 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 5 5 1 1 5 5 5 5 5 1 5 1
 5 5 5 1 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 5 5 5 5 5 5 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 7 7 2 2 3 1 5 5 5 0 4 3 5 0 3 3 0 0 3 4 3 3 3 3 5 5 3 3 5 1 1 1
 1 5 3 3 5 1 1 4 3 3 1 1 1 5 5 5 1 1 1 5 3 4 4 4 5 5 5 0 3 0 5 5 0 3 3 5 7
 2 2 5 5 1 1 5 0 0 0 5 5 1 1 1 1 1 1 5 3 3 5 1 1 1 1 1 1 1 1 1 1 1 1 1 5 4
 3 5 1 5 5 5 5 5 5 5 5 0 3 3 5 5 3 3 4 4 3 5 1 5 5 5 1 1 1 1 5 0 3 3 1 4 3
 3 1 1 1 1 5 5 5 1 5 5 3 5 3 5 3 3 4 3 3 5 0 6 6 6 6 0 0 0 3 0 7 2 4 5 4 2
 2 4 3 0 0 3 3 1 5 1 5 1 1 1 1 1 1 1 1 1 1 1 5 5 5 5 3 3 5 1 3 4 2 3 4 3 4
 3 3 5 5 4 0 4 4 4 4 3 3 

In [ ]:
kmeans_iq = KMeans(random_state=rng).fit(iq_train_features)
clusters_iq = kmeans_iq.predict(iq_train_features)
